## ModOAP - Statistiques sur les images contenues dans un ou plusieurs documents Gallica

Ce script permet de calculer des statistiques sur les images contenues dans un ou plusieurs documents Gallica.

**Calculs effectués :**  
  - Nombre d'images par document et par position dans le document (1er tiers, 2nd tiers, 3e tiers)
  - Nombre d'images par document et par position dans la page (moitié haut, moitié bas, moitié gauche, moitié droite)
  - Taux d'images par page par document
  - Taille moyenne des images par document (exprimée en pixels)
  - Pourcentage moyen de couverture de la page par document
  - Taux d'images par page par année du corpus total

**Fonctionnement :**
1. Récupérer les blocs images d'un ou plusieurs documents (au format structuré JSON) en spécifiant les liens ARK des documents hébergés sur Gallica.
2. Lancer le calcul statistique. Les résultats sont présentés sous forme de graphiques dans un fichier html de visualisation, généré par ce script.

In [20]:
 #@markdown ### Préparation et connexion à un compte Google Drive
#@markdown Lancer cette cellule, puis accepter la connexcion à un compte Google Drive si demandé.

import requests
from openpyxl import load_workbook
import urllib.request, urllib.error, urllib.parse
from urllib.error import HTTPError, URLError
import os
import json
from bs4 import BeautifulSoup
try :
  import xmltodict
except :
  !pip -q install xmltodict
  import xmltodict
import unicodedata
import re 
import glob
from tqdm import tqdm
import statistics   
from google.colab import drive



def remove_accents(s):
  # In : chaine avec caractères diachrités
  # Out : chaine sans accent
  return ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

def normalisation_titre(titre):
  # In : chaine titre
  # Out : chaine titre pour nom de fichier
  titre = remove_accents(titre)
  titre = re.sub('[^a-zA-Z0-9- ]', '', titre)
  titre = re.sub('[ ]', '_', titre)
  return "_".join(titre.split("_")[:6])

def Orientation(w,h):
  if w > h :
    return "horizontale"
  elif w < h :
    return "verticale"
  elif w == h :
    return "square"

def Tiers_document(page,premier_tiers,deuxieme_tiers):
  if page <= premier_tiers :
    return 1
  elif premier_tiers < page <= deuxieme_tiers :
    return 2
  elif page > deuxieme_tiers :
    return 3

def Moitie_page(hpos, vpos, page_hauteur, page_largeur):
  if vpos > page_hauteur / 2 :
    hb = "h"
  else :
    hb = "b"
  if hpos < page_largeur / 2:
    gd = "g"
  else : 
    gd = "d"
  return hb, gd

def get_var_tb(tb) :
  # In : textblock (ordered dict key, dict)
  # récupère la position et l'id du bloc texte
  # lance get_var_tb
  width = tb["@WIDTH"]
  height = tb["@HEIGHT"]
  hpos = tb["@HPOS"]
  vpos = tb["@VPOS"]
  id = tb["@ID"]
  return width, height, hpos, vpos, id

def get_tb(tb_entry) :
  # In : textblock (ordered dict key, list ou dict)
  # récupère la position et l'id du bloc texte
  # lance get_var_tb
  if isinstance(tb_entry, list) :
    for tb in tb_entry :
      width, height, hpos, vpos, id = get_var_tb(tb)
  else : 
    width, height, hpos, vpos, id = get_var_tb(tb_entry)
  return width, height, hpos, vpos, id

def block_treatment(tb, num_page) :
  # In :  textblock (ordered dict key), numéro de page (int)
  # lance get_tb
  # ajoute la position et le contenu du bloc texte dans un dictionnaire
  width, height, hpos, vpos, id = get_tb(tb)
  position_dic = {"width" : width, "height" : height, "hpos" : hpos, "vpos" : vpos}
  dico_id = {"Page_Num" : num_page, "Position" : position_dic}
  text_blocks[id] = dico_id


def infos_doc(ark):
  # In :  identifiant ark
  # Out : titre, date de publication du document (str)
  url_biblio = "https://gallica.bnf.fr/services/OAIRecord?ark="+ark
  s = requests.get(url_biblio, stream=True)
  bibliodico = xmltodict.parse(s.text)
  try :
    titre = bibliodico["results"]["title"]
  except :
    titre = "unknown"
  try :
    date_pub = bibliodico["results"]["date"]["#text"]
  except :
    date_pub = "unknown"
  return titre, date_pub
  
def nombre_pages(ark):
  # In :  identifiant ark
  # Out : nombre de pages (int)
  PAGINATION_BASEURL = 'https://gallica.bnf.fr/services/Pagination?ark='
  url = "".join([PAGINATION_BASEURL, ark])
  s = requests.get(url, stream=True)
  paginationdic = xmltodict.parse(s.text)
  nb_pages = int(paginationdic["livre"]["structure"]["nbVueImages"])
  return nb_pages

def ark_processing(ark) :
  # In :  identifiant ark
  # Out : fichier json contenant : 
    # titre, date, nombre de pages du document
    # id, position, contenu des blocs de textes de chaque page
  flag = 0
  titre, date_pub = infos_doc(ark)
  titre_fichier = normalisation_titre(titre)  
  try :
    nombre_pagess = nombre_pages(ark)
  except :
    print("Pagination indisponible, document non-téléchargé : ", ark)

  print("Titre du document : ", titre)
  print("Dossier de destination : ", destination_dir)
  print("Référencement des images du document :")
  info_doc = {"Titre" : titre, "Publication_Date" : date_pub, "Total_Pages" : nombre_pagess }
  global text_blocks 
  text_blocks = {}

  # Pour chaque page du document :
  for num_page in tqdm(range(1,nombre_pagess+1)) :
  #for num_page in range(1,10) : 
    # Transforme le fichier OCR ALTO de la page en un dictionnaire :
    alto_url = 'https://gallica.bnf.fr/RequestDigitalElement?O='+ark+'&E=ALTO&Deb='+str(num_page)
    s = requests.get(alto_url, stream=True)
    altodic = xmltodict.parse(s.text)

    if flag == 0 :
      info_doc["Page_hauteur"] = altodic["alto"]["Layout"]["Page"]["@HEIGHT"]
      info_doc["Page_largeur"] = altodic["alto"]["Layout"]["Page"]["@WIDTH"]
      flag = 1
    else : pass

    # Si un ou plusieurs blocs de texte sont directement présents dans le PrintSpace : 
 
    
    try :
      tb_entry = altodic['alto']["Layout"]["Page"]["PrintSpace"]["Illustration"]
      if isinstance(tb_entry, list) :
        for tb in tb_entry :
          block_treatment(tb, num_page)
        
      else :
        block_treatment(tb_entry, num_page)
    except :
      pass
    # Si un ou plusieurs blocs composés sont présents dans le PrintSpace : 
    try :
      cb_entry = altodic['alto']["Layout"]["Page"]["PrintSpace"]["ComposedBlock"]
      print("cbentry")
      if isinstance(cb_entry, list) :
        for cb in cb_entry :
          print("cbentry")
          tb_entry = cb["Illustration"]
          if isinstance(tb_entry, list) :
            for tb in tb_entry :
              block_treatment(tb, num_page)
          else :
            block_treatment(tb_entry, num_page)
      else :
        tb_entry = cb_entry["Illustration"]
        if isinstance(tb_entry, list) :
          for tb in tb_entry :
            block_treatment(tb, num_page)
        else :
          block_treatment(tb_entry, num_page)
    except :
      pass
  # Création du fichier final :
  jsondic = {"Infos_Doc" : info_doc, "Text_Blocks" : text_blocks} 
  with open(os.path.join(destination_dir,str(date_pub)+"_"+titre_fichier+"_"+ark+"_blocs_images.json"), 'w') as jout :
    json.dump(jsondic, jout)
  print("Le texte du document a été sauvegardé dans le fichier ", os.path.join(destination_dir,str(date_pub)+"_"+titre_fichier+"_"+ark+"_blocs_images.json"))

def bnf2gall(arkbnf):
  # In :  identifiant ark au format type cb453908509
  # Out : identifiant ark au format type bpt6k9799524x (consultable sur Gallica)
  url = "https://catalogue.bnf.fr/ark:/12148/"+str(arkbnf)
  s = requests.get(url, stream=True)
  html = BeautifulSoup(s.content) # html5lib
  for link in html.findAll('a', {'class': 'exemplaire-action-visualiser'}):
    ark = link['href'].split("/")[-1]
  return ark


# chargement d'un google drive
if not os.path.exists("/content/drive/MyDrive/") :
  drive.mount('/content/drive/')

In [ ]:
#@markdown ### Préparation du corpus
#@markdown 1. Entrer un chemin vers un dossier (créé si inexistant) situé sur un Google Drive où télécharger le corpus (documents au format JSON), puis lancer la cellule.

destination_dir = ""#@param {type:"string"}

#@markdown Exemple de chemin : /content/drive/MyDrive/stats_images_gallica


#@markdown 2. Entrer un identifiant ARK unique pour un calcul sur un seul document, ou le chemin vers un fichier txt comprenant plusieurs liens ARK (un lien par ligne)
arks = ["bpt6k2036384", "bpt6k203639h", "bpt6k203640f", "bpt6k203641t"]

arks = ""#@param {type:"string"}

#@markdown Exemple : bpt6k2036384 ou /content/drive/MyDrive/stats_images_gallica/id_arks.txt

if not os.path.exists(destination_dir) :
  os.makedirs(destination_dir)
arks = arks.strip()
if arks.endswith(".txt") :
  liste_arks = []
  with open(arks, "r") as arkin :
    for ark in arkin.readlines():
      ark = ark.strip()
      liste_arks.append(ark)
elif "." in arks :
  print("Problème dans le champ arks : entrer un identifiant ARK ou le chemin absolu vers un fichier .txt")

else :
  liste_arks = [arks]

print("identifiants repérés : ",liste_arks)

for ark in liste_arks :
  ark_processing(ark)

In [ ]:
#@markdown ### Calcul statistique

#@markdown 1. Entrer le chemin vers le dossier contenant les documents du corpus au format JSON : 
destination_dir = ""#@param {type:"string"}

#@markdown Exemple de chemin : /content/drive/MyDrive/stats_images_gallica


#@markdown 2. Entrer un nom sans extension pour le fichier de résultats à générer, puis lancer la cellule.
fichier_resultats = "" #@param {type:"string"}


jsons = [f for f in glob.glob(os.path.join(dossier_travail,"*.json"))]

print("Nombre de documents dans le corpus : ",len(jsons))
graphes1 = []
graphes2 = []
graphes3 = []
graphes4 = []
graphes5 = []
graphes6 = []
corpus=[]
for j in jsons :
  with open(j,"r") as jin :
    dico = json.load(jin)

    # a l'échelle de l'image : 
      # pourcentage de la page

    orientations = []
    tailles = []
    pct_pages = []
    tiers_documents = []
    hbs = []
    gds = []


    for image in dico["Text_Blocks"].keys() :
      page = dico["Text_Blocks"][image]["Page_Num"] 
      width = int(dico["Text_Blocks"][image]["Position"]["width"])
      height = int(dico["Text_Blocks"][image]["Position"]["height"])
      hpos = int(dico["Text_Blocks"][image]["Position"]["hpos"])
      vpos = int(dico["Text_Blocks"][image]["Position"]["vpos"])
      page_hauteur = int(dico["Infos_Doc"]["Page_hauteur"])
      page_largeur = int(dico["Infos_Doc"]["Page_largeur"])
      taille_page = page_hauteur * page_largeur
      premier_tiers = dico["Infos_Doc"]["Total_Pages"] / 3
      deuxieme_tiers = premier_tiers * 2


      taille_image = width * height
      tailles.append(taille_image)
      orientation = Orientation(width, height)
      orientations.append(orientation)
      pct_page = taille_image / taille_page * 100
      pct_pages.append(pct_page)
      tiers_document = Tiers_document(page,premier_tiers,deuxieme_tiers)
      tiers_documents.append(tiers_document)
      hautbas, gauchedroite = Moitie_page(hpos, vpos, page_hauteur, page_largeur)
      hbs.append(hautbas)
      gds.append(gauchedroite)
      titre = normalisation_titre(dico["Infos_Doc"]["Titre"])
      corpus.append(dico["Infos_Doc"]["Titre"])

    nb_images = len(orientations)
    taux_images_page = nb_images / int(dico["Infos_Doc"]["Total_Pages"])
    taille_moy_images = statistics.mean(tailles)
    pct_pages_moy = statistics.mean(pct_pages)

    # graph 1 : 
    graph1 = [titre,nb_images,tiers_documents.count(1),tiers_documents.count(2),tiers_documents.count(3)]
    graphes1.append(graph1)

    graph2 = [titre,nb_images,hbs.count("h"),hbs.count("b"),gds.count("g"),gds.count("d")]
    graphes2.append(graph2)

    graph3 = [titre,taux_images_page]
    graphes3.append(graph3)

    graph5 = [titre,taille_moy_images]
    graphes5.append(graph5)

    graph6 = [titre,pct_pages_moy]
    graphes6.append(graph6)   

    graph4 = [dico["Infos_Doc"]["Publication_Date"][:4],taux_images_page]
    graphes4.append(graph4)

graphe1 = "function drawgraphe1() {var data = new google.visualization.DataTable();data.addColumn(\'string\', \'Document\');data.addColumn(\'number\', \'Nombre images\');data.addColumn(\'number\', \'Nombre Images Tiers 1\');data.addColumn(\'number\', \'Nombre Images Tiers 2\');data.addColumn(\'number\', \'Nombre Images Tiers 3\');data.addRows("+str(graphes1)+");var options = {title: \'Nombre dimages par document et position dans le document\',bar: { groupWidth: \"90%\" }};var chart = new google.visualization.ColumnChart(document.getElementById(\'chart_div1\'));chart.draw(data, options);}"
graphe2 = "function drawgraphe2() {var data = new google.visualization.DataTable();data.addColumn(\'string\', \'Document\');data.addColumn(\'number\', \'Nombre images\');data.addColumn(\'number\', \'Nombre Images Moitié Haut\');data.addColumn(\'number\', \'Nombre Images Moitié Bas\');data.addColumn(\'number\', \'Nombre Images Moitié Gauche\');data.addColumn(\'number\', \'Nombre Images Moitié Droite\');data.addRows("+str(graphes2)+");var options = {title: \'Nombre dimages par document et position dans la page\',bar: { groupWidth: \"90%\" }};var chart = new google.visualization.ColumnChart(document.getElementById(\'chart_div2\'));chart.draw(data, options);}"
graphe3 = "function drawgraphe3() {var data = new google.visualization.DataTable();data.addColumn(\'string\', \'Document\');data.addColumn(\'number\', \'Taux images par page\');data.addRows("+str(graphes3)+");var options = {title: \'Taux images par page par document\',bar: { groupWidth: \"90%\" }};var chart = new google.visualization.ColumnChart(document.getElementById(\'chart_div3\'));chart.draw(data, options);}"

graphe6 = "function drawgraphe6() {var data = new google.visualization.DataTable();data.addColumn(\'string\', \'Document\');data.addColumn(\'number\', \'Pourcentage moyen de couverture de la page\');data.addRows("+str(graphes6)+");var options = {title: \'Pourcentage moyen de couverture de la page par document\',bar: { groupWidth: \"90%\" }};var chart = new google.visualization.ColumnChart(document.getElementById(\'chart_div6\'));chart.draw(data, options);}"
graphe5 = "function drawgraphe5() {var data = new google.visualization.DataTable();data.addColumn(\'string\', \'Document\');data.addColumn(\'number\', \'Taille moyenne des images\');data.addRows("+str(graphes5)+");var options = {title: \'Taille moyenne des images par document\',bar: { groupWidth: \"90%\" }};var chart = new google.visualization.ColumnChart(document.getElementById(\'chart_div5\'));chart.draw(data, options);}"

graphe4 = "function drawgraphe4() {var data = new google.visualization.DataTable();data.addColumn(\'string\', \'Annee\');data.addColumn(\'number\', \'Taux images par page\');data.addRows("+str(graphes4)+");var options = {title: \'Taux Images par page par année\'};var chart = new google.visualization.LineChart(document.getElementById(\'chart_div4\'));chart.draw(data, options);}"


with open(os.path.join(destination_dir,fichier_resultats+".html"), "w") as htmlout : 
  htmlout.write("<!DOCTYPE html><html><head><meta charset=\"utf-8\"><title>Visualisation Statistiques</title></head><body><script type=\"text/javascript\" src=\"https://www.gstatic.com/charts/loader.js\"></script><script type=\"text/javascript\">google.charts.load(\'current\', {packages: [\'corechart\', \'bar\']});google.charts.setOnLoadCallback(drawgraphe1);google.charts.setOnLoadCallback(drawgraphe2);google.charts.setOnLoadCallback(drawgraphe3);google.charts.setOnLoadCallback(drawgraphe4);google.charts.setOnLoadCallback(drawgraphe5);google.charts.setOnLoadCallback(drawgraphe6);"+graphe1+graphe2+graphe3+graphe4+graphe5+graphe6+"</script><h2>Statistiques sur les images du corpus</h2><br><p>Composition du corpus :<br>"+str(set(corpus))+"</p><div id=\"chart_div1\"></div><div id=\"chart_div2\"></div><div id=\"chart_div3\"></div><div id=\"chart_div5\"></div><div id=\"chart_div6\"></div><div id=\"chart_div4\"></div></body></html>")

print("Les résultats ont été générés dans le fichier",os.path.join(destination_dir,fichier_resultats+".html"))